# Explore the Linkedin API
https://pypi.python.org/pypi/python-linkedin/4.0

In [ ]:
import os
import json
import requests

from linkedin import linkedin

import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets

In [ ]:
# store -r access_token
access_token = os.getenv('LINKEDIN_ACCESS_TOKEN')
application = linkedin.LinkedInApplication(token=access_token)
# access_token

`application` object can be used to run querries as per, http://ozgur.github.io/python-linkedin/

https://developer.linkedin.com/docs/fields/company-profile

https://developer.linkedin.com/docs/fields/positions

https://developer.linkedin.com/docs/fields/companies

https://developer.linkedin.com/docs/guide/v2/concepts/paging

In [ ]:
rspnse = application.search_company(
        selectors=[{'companies': ['id',
                                  'name',
                                  'universal-name',
                                  {'company-type': 'name'},
                                  'email-domains',
                                  'website-url',
                                  {'industries': 'name'},
                                  {'status': 'name'},
                                  'twitter-id',
                                  {'employee-count-range': 'code'},
                                  'specialties',
                                  'description',
                                  'locations',
                                  'founded-year',
                                  'num-followers',
                                 ]
        }],
        params={'start': 0, 'count': 1, 'keywords': 'data'}
    )
r = json.dumps(rspnse['companies']['values'])
pd.io.json.json_normalize(json.loads(r))

In [ ]:
# Search URL is https://api.linkedin.com/v1/company-search:(companies:(name,universal-name,website-url))?start=0&count=10&keywords=apple%20microsoft
companies_df = pd.DataFrame([])
i = 0
while i <= 5000:
    response = application.search_company(
        selectors=[{'companies': ['id',
                                  'name',
                                  'universal-name',
                                  {'company-type': 'name'},
                                  # 'email-domains',
                                  'website-url',
                                  {'industries': 'name'},
                                  {'status': 'name'},
                                  'twitter-id',
                                  {'employee-count-range': 'code'},
                                  {'specialties': 'values'},#
                                  'description',
                                  {'locations': 'values'},#
                                  'founded-year',
                                  'num-followers',
                                 ]
        }],
        params={'start': i, 'count': i + 20, 'keywords': 'data'}
    )
    r = json.dumps(response['companies']['values'])
    df = pd.io.json.json_normalize(json.loads(r))
    companies_df = companies_df.append(df, ignore_index=True)
    i += 20
companies_df.shape

Some inconsistency in the data is breaking the loop ^ after < 2k records returned

In [ ]:
companies_df = pd.to_json("c_empl_cnt_df.json")
companies_df.to_csv("c_empl_cnt_df.csv", encoding='utf-8')

engine = create_engine('postgresql://uname@localhost:5432/mydatabase')
companies_df.to_sql('Linkedin_Companies_Employee_Count_C', engine)
companies_df.shape

In [ ]:
c_empl_cnt_df = companies_df[companies_df['employeeCountRange.code']=='C']
c_empl_cnt_df.shape